# constructing 2D/3D local box using GeoPoints.jl 

Nobuaki Fuji (IPGP/UPC/IUF) December 2025

you can also see how to use lazyProduceOrLoad, which is not safe solution but when debugging, it's cool! (but don't write a biiiig jdl2 neither!)

In [ ]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .commonBatchs, .planet1D, .GeoPoints
using Colors

In [ ]:
set_default_planet!(:Earth) # if you wish to go to other telluric bodies, otherwise this is not necessary

In [ ]:
# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(48.8566,2.3522) # Paris (latitude, longitude)
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)


Δx = 100.0 # in metre
Δz = 100.0

altMax = 10.e3 # in metre
altMin = -100.e3 # in metre

# altMax and altMin are measured from the middle point at p1 and p2 (but the planet's surface normally), the user can change the option hidden in constructLocalBox

In [ ]:
@show p1 # GeoPoint has some attributes

In [ ]:
# make a box/rectangle

allGridsInGeoPoints, allGridsInCartesian, effectiveRadii=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax)
#allGridsInGeoPoints, allGridsInCartesian, effectiveRadii=lazyProduceOrLoad("boxGrids",constructLocalBox,p1,p2,Δx,Δz,altMin,altMax) <- don't do this, it's too heavy


In [ ]:
allGridsInCartesian[20,30]

In [ ]:
#seismicModel=getParamsAndTopo(allGridsInGeoPoints,effectiveRadii,2.0) # this can be also GPUed
seismicModel=lazyProduceOrLoad("seismicModel",getParamsAndTopo,allGridsInGeoPoints,effectiveRadii,2.0)
#seismicModel=lazyProduceOrLoad("seismicModel") # this is the laziest way to load

In [ ]:
Nx,Nz=size(seismicModel.ρ)

In [ ]:
maximum(seismicModel.ρ)

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in allGridsInCartesian[:,1]]*1.e-3
zvals = [p.xz[2] for p in allGridsInCartesian[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModel.ρ;
    colormap = :seismic,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "density model")
)
Colorbar(fig[1,2], hm, label="density")
fig

# if you want to put some 2D/3D perturbation into it (in percent or absolute value, as you wish)


In [ ]:
imageFilePer="./myPerturbation.png"
modelPer= read2DimageModel(imageFilePer; Ncolor=256, colorbar = [RGB(0.0, 0.0, 0.0), RGB(1.0, 1.0, 1.0)] ,values = [0.0,1.0])

# here is the size of the region to be perturbed
nx,nz=size(seismicModel.Vpv)
VpvSub = seismicModel.Vpv[Int(0.2*nx):Int(0.8*nx),Int(0.2*nz):Int(0.8*nz)]
newPer = adjustArray(VpvSub,modelPer)
@. VpvSub = 0.8*VpvSub + 0.4*newPer*VpvSub

seismicModel.Vpv[Int(0.2*nx):Int(0.8*nx),Int(0.2*nz):Int(0.8*nz)]= VpvSub[:,:]

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in allGridsInCartesian[:,1]]*1.e-3
zvals = [p.xz[2] for p in allGridsInCartesian[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModel.Vpv;
    colormap = :seismic,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "Vpv model")
)
Colorbar(fig[1,2], hm, label="Vpv")
fig

# 3D box ? yes

In [ ]:
p1 = GeoPoint(48.8566,2.3522) # Paris (latitude, longitude)
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)


Δx = 500.0 # in metre
Δy = 500.0
Δz = 500.0

altMax = 20.e3 # in metre
altMin = -100.e3 # in metre

horizontalDepth = 50.e3

In [ ]:

allGridsInGeoPoints3D, allGridsInCartesian3D, effectiveRadii3D=constructLocalBox(p1,p2,Δx,Δy,Δz,-horizontalDepth,horizontalDepth,altMin,altMax)

#ok 3D box version needs to be GPUed

In [ ]:
#seismicModel3D=getParamsAndTopo(allGridsInGeoPoints3D,effectiveRadii3D,2.0)
seismicModel3D=lazyProduceOrLoad("seismicModel3D",getParamsAndTopo,allGridsInGeoPoints3D,effectiveRadii3D,2.0)
#seismicModel3D=lazyProduceOrLoad("seismicModel3D")

In [ ]:
using GLMakie
GLMakie.activate!()
Makie.inline!() 
Nx3D,Ny3D,Nz3D=size(allGridsInCartesian3D)

#x = (0:Nx3D-1) .* Δx .* 1e-3
#y = (0:Ny3D-1) .* Δy .* 1e-3 .- horizontalDepth*1.e-3
#z = (0:Nz3D-1) .* Δz .* 1e-3 .+ altMin*1.e-3

x = [p.xyz[1] for p in allGridsInCartesian3D[:,1,1]]*1.e-3
y = [p.xyz[2] for p in allGridsInCartesian3D[1,:,1]]*1.e-3
z = [p.xyz[3] for p in allGridsInCartesian3D[1,1,:]]*1.e-3
A = seismicModel3D.Vpv

f = Figure()
ax = Axis3(f[1, 1])

volume!(ax,
    x[1] .. x[end],
    y[1] .. y[end],
    z[1] .. z[end],
    seismicModel3D.Vpv,
    algorithm = :absorption,   # optional, makes it nicer
    colormap = :viridis
)
f

# another way of importing model parameters

## we can read images or cartoons

In [ ]:
modelName="marmousi"
imageFile="../dataInput/model/random/marmousi.png"
modelDefinitionMethod="2DimageFile" # ToyModel or 2DimageFile (or 1DsphericalPlanet)
model=defineModel(imageFile)

model construction 

In [ ]:
#s
_, allGridsInCartesianMar, _ = constructLocalBox(model,0.0,34.0,-40.0,0.0)
seismicModelMarmousi = makeAdHocSeismicModel(model, 1.0, 2.8, 1.5, 5.5, 0.0, 3.2)

#constructLocalBox for marmousi models should be written!

In [ ]:
allGridsInCartesianMar[1,100].xz[2]

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in allGridsInCartesianMar[:,1]]*1.e-3
zvals = [p.xz[2] for p in allGridsInCartesianMar[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModelMarmousi.Vsh;
    colormap = :seismic,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth", title = "Vsh model")
)
Colorbar(fig[1,2], hm, label="Vsh")
fig

# now construction of OPT operators

## first we give the equation to be solved with famousEquation or hand-made PDEs

In [ ]:
famousEquationType="2DacousticTime"
exprs,fields,vars,extexprs,extfields,extvars,coordinates,∂,∂²=famousEquations(famousEquationType)

# Let's go to Mars

In [ ]:
set_default_planet!(:Mars)


# ok i need to change how to call 1D planet models too (because it is already called by DSM1D and it is not very much flexible but here I just use the same params as Earth)

In [ ]:
p1 = GeoPoint(15.0,135.0) # we should see Elysium planitia at least
p2 = GeoPoint(15.0,180.0) # 

In [ ]:
Δx = 3.e3 # in metre
Δy = 3.e3
Δz = 3.e3

horizontalDepthMin = -1000.e3
horizontalDepthMax = 1000.e3


altMax = 100.e3 # in metre
altMin = -600.e3 # in metre



In [ ]:
allGridsInGeoPoints, allGridsInCartesian, effectiveRadii=constructLocalBox(p1,p2,Δx,Δy,Δz,horizontalDepthMin,horizontalDepthMax,altMin,altMax)

In [ ]:
Nx,Ny,Nz=size(allGridsInCartesian)

In [ ]:
seismicModel=getParamsAndTopo(allGridsInGeoPoints,2.0)

In [ ]:
using CairoMakie
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.Vsv[:,(Ny÷6)*1,:];
    colormap = :seismic,
    colorrange=(0,15),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "Vph model")
)
ylims!(ax,-200,300)
xlims!(ax,0,400)
Colorbar(fig[1,2], hm, label="P-wave")
fig